In [3]:
import pickle
import tflearn
import tensorflow as tf
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import json
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [4]:
stemmer =LancasterStemmer()

with open('E:\TY SEM 2\EDI\edai\chatbot\intents.json', 'r', encoding='utf8') as file:
    data = json.load(file)


In [5]:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    ignore_wrods = ['?', '.', ',', '\'', ';' ':']
    for intent in data['intents']:
        for pattern in intent['patterns']:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [6]:
    words = [stemmer.stem(w.lower()) for w in words if w not in ignore_wrods]

    words = sorted(list(set(words)))
    labels = sorted(labels)

In [7]:
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

In [8]:
    for x, doc in enumerate(docs_x):
        bag = []
        wrds = [stemmer.stem(w.lower()) for w in doc]
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)
        with open("data.pickle", "wb") as f:
            pickle.dump((words, labels, training, output), f)

In [9]:
training = np.array(training)
output = np.array(output)


In [10]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 6999  | total loss: 0.25291 | time: 0.027s
| Adam | epoch: 1000 | loss: 0.25291 - acc: 0.9893 -- iter: 48/50
Training Step: 7000  | total loss: 0.22827 | time: 0.032s
| Adam | epoch: 1000 | loss: 0.22827 - acc: 0.9904 -- iter: 50/50
--
INFO:tensorflow:e:\TY SEM 2\EDI\edai\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)

In [13]:
def chat(inp):
    results = model.predict([bag_of_words(inp, words)])
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
        if tg['tag'] == tag:
            responses = tg['responses']

    return(random.choice(responses))

In [14]:
response = chat("How is Life at VIT")
response

Start talking with the bot (type quit to stop)!


'Well if you are admitted to VIT pune, itself it means you are in one of the prestigious college among others in Pune and Maharastra as well.\n\nThe day that is spent in a college depends on which module are scheduled to attend the lectures. There are two modules schedule in vit, one in morning and other being in the afternoon. Morning lectures starts at 8:00 am and the college lectures including lab practicals lasts untill 12–1 pm. And if you are at afternoon module then the college starts at 1 pm and lasts untill 5–6 pm. This is the college timing of VIT (Saturday and Sunday off). There is free time after every two lectures and generally not more than 3 lectures are scheduled for the day.\n\nFor free time there are canteens,reading room, IT labs and libraries and the boat club. After two lectures generally there is gap of 1 hr for next lecture students used to sit at the boat club usually with friends. Or there some other places where one can spend some time with friends. If anyone i